# 1. Introduction
**FastText** is an open-source, text classification library created by **Facebook's AI Research (FAIR)** team. It provides an efficient and fast way to classify text data.

A *FastText News Classifier* is a machine learning model that leverages **FastText** to classify news articles into different categories (such as sports, politics, entertainment, etc.).

**Thanks to:**
* [E-commerce Text Classification Using FastText](https://www.kaggle.com/code/sunilthite/e-commerce-text-classification-using-fasttext/input)
* [Loading Kaggle data directly into Google Colab](https://www.youtube.com/watch?v=yEXkEUqK52Q)
* [Importing Datasets from Kaggle to Google Colab](https://saturncloud.io/blog/importing-datasets-from-kaggle-to-google-colab/)
* [kaggle dataset download 403 Forbidden](https://stackoverflow.com/questions/75569191/kaggle-dataset-download-403-forbidden)
* [Traffic Accident in Indonesia](https://www.kaggle.com/datasets/dodyagung/accident)
* [Jigsaw Regression Based Data](https://www.kaggle.com/datasets/nkitgupta/jigsaw-regression-based-data)

# 2. Import libraries

In [1]:
!pip install fasttext

In [2]:
# FastText library for text classification and training the FastText model.
import fasttext

# pandas library for data manipulation.
import pandas as pd

# 3. Load the dataset
Loads the dataset from a CSV file into a DataFrame for easy manipulation

In [3]:
# Kaggle library to download datasets from Kaggle
!pip install kaggle

In [4]:
# Mount Google Drive: Import the Drive to access and store the Kaggle API key in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Set Kaggle Configuration: To direct Kaggle to the appropriate directory in Drive
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

In [6]:
# copy the Kaggle api key to the google drive
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
# give the permission to the JSON file
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
# download the dataset from Kaggle using the Kaggle API.

# !kaggle datasets download -d nkitgupta/jigsaw-regression-based-data
# !unzip /content/jigsaw-regression-based-data.zip
# !rm /content/jigsaw-regression-based-data.zip

!kaggle datasets download -d dodyagung/accident
# !unzip /content/accident.zip
# !rm /content/accident.zip

Dataset URL: https://www.kaggle.com/datasets/dodyagung/accident
License(s): CC-BY-NC-SA-4.0
accident.zip: Skipping, found more recently modified local copy (use --force to force download)


In [11]:
df = pd.read_csv('/content/accident.csv')
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/content/accident.csv'